In [ ]:
%load_ext jupyflink.magics

In [ ]:
%flink_gateway_init --debug

In [ ]:
%%flink_gateway_sql_prepare
ADD JAR '/opt/gateway/jars/flink-sql-connector-kafka-3.2.0-1.19.jar';

In [ ]:
%%flink_gateway_sql_prepare
drop table if exists attributes;
CREATE TABLE attributes (
    `id` STRING,
    `parentId` STRING,
    `entityId` STRING,
    `name` STRING,
    `nodeType` STRING,
    `valueType` STRING,
    `type` STRING,
    `attributeValue` STRING,
    `datasetId` STRING,
    `unitCode` STRING,
    `deleted` BOOLEAN,
    `synced` BOOLEAN,
    `ts` TIMESTAMP(3) METADATA FROM 'timestamp' VIRTUAL,
    WATERMARK FOR ts AS ts
) WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild.attributes',
  'json.fail-on-missing-field' = 'False',
  'json.ignore-parse-errors' = 'True',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
);

DROP VIEW IF EXISTS `attributes_view`;
CREATE VIEW `attributes_view` AS
SELECT
`id`,
`parentId`,
`entityId`,
`name`,
`nodeType`,
`valueType`,
`type`,
 `attributeValue`,
`datasetId`,
`deleted`,
`synced`,
`ts` FROM (
  SELECT *,
ROW_NUMBER() OVER (PARTITION BY `id`, `datasetId`
ORDER BY ts DESC) AS rownum
FROM `attributes` )
WHERE rownum = 1;

drop table if exists attributes_insert;
CREATE TABLE attributes_insert (
    `id` STRING,
    `parentId` STRING,
    `entityId` STRING,
    `name` STRING,
    `nodeType` STRING,
    `valueType` STRING,
    `type` STRING,
    `attributeValue` STRING,
    `datasetId` STRING,
    `unitCode` STRING,
    `deleted` BOOLEAN,
    `synced` BOOLEAN,
    `ts` TIMESTAMP(3) METADATA FROM 'timestamp' VIRTUAL,
    WATERMARK FOR ts AS ts,
    PRIMARY KEY (`id`, `datasetId`) NOT ENFORCED
) WITH (
  'topic' = 'iff.ngsild.attributes',
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092'
);

drop table if exists attributes_writeback;
CREATE TABLE attributes_writeback (
    `id` STRING,
    `parentId` STRING,
    `entityId` STRING,
    `name` STRING,
    `nodeType` STRING,
    `valueType` STRING,
    `type` STRING,
    `attributeValue` STRING,
    `datasetId` STRING,
    `unitCode` STRING,
    `deleted` BOOLEAN,
    `synced` BOOLEAN,
    PRIMARY KEY (`id`, `datasetId`) NOT ENFORCED
) WITH (
  'topic' = 'iff.ngsild.attributes',
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092'
);

In [ ]:
%%flink_gateway_sql_prepare
drop table if exists ngsild_updates;
CREATE TABLE ngsild_updates (
  `op` STRING,
  `overwriteOrReplace` Boolean,
  `noForward` Boolean,
  `entities` STRING
) WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild-updates',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'latest-offset',
  'format' = 'json'
);

In [ ]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `alerts`;
CREATE TABLE `alerts` (
    `resource` STRING,
    `event` STRING,
    `environment` STRING,
    `service` ARRAY < STRING >,
    `severity` STRING,
    `customer` STRING,
    `text` STRING,
    PRIMARY KEY (resource, event) NOT ENFORCED
) WITH (
    'connector' = 'kafka',
    'format' = 'json',
    'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'scan.startup.mode' = 'latest-offset',
    'topic' = 'iff.alerts'
);

DROP TABLE IF EXISTS `alerts_bulk`;
CREATE TABLE `alerts_bulk` (
    `resource` STRING,
    `event` STRING,
    `environment` STRING,
    `service` ARRAY < STRING >,
    `severity` STRING,
    `customer` STRING,
    `text` STRING,
    watermark FOR ts AS ts - INTERVAL '0.0' SECONDS,
    `ts` TIMESTAMP(3) METADATA
    FROM
        'timestamp' VIRTUAL,
        PRIMARY KEY (resource, event) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'iff.alerts.bulk'
);

DROP VIEW IF EXISTS `alerts_bulk_view`;



In [ ]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `entities`;
CREATE TABLE `entities` (
    `id` STRING,
    `type` STRING,
    `deleted` BOOLEAN,
    `ts` TIMESTAMP(3) METADATA
    FROM
        'timestamp',
        watermark FOR `ts` AS `ts`
) WITH (
    'connector' = 'kafka',
    'format' = 'json',
    'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'scan.startup.mode' = 'latest-offset',
    'topic' = 'iff.ngsild.entities'
);
DROP VIEW IF EXISTS `entities_view`;
CREATE VIEW `entities_view` AS
SELECT
    `type`,
     `id`,
     `deleted`,
     `ts`
FROM
    (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY `id` ORDER BY ts DESC) AS rownum FROM `entities`
    ) WHERE rownum = 1;
DROP TABLE IF EXISTS `rdf`;
CREATE TABLE `rdf` (
    `subject` STRING,
    `predicate` STRING,
    `object` STRING,
    `index` INTEGER,
    PRIMARY KEY (subject, predicate, index) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'iff.rdf'
);

In [ ]:
%%flink_gateway_sql_prepare 
DROP TABLE IF EXISTS `constraint_table`;
CREATE TABLE `constraint_table` (
    `id` INTEGER,
    `targetClass` STRING,
    `propertyPath` STRING,
    `subpropertyPath` STRING,
    `propertyClass` STRING,
    `propertyNodetype` STRING,
    `attributeType` STRING,
    `maxCount` STRING,
    `minCount` STRING,
    `severity` STRING,
    `minExclusive` STRING,
    `maxExclusive` STRING,
    `minInclusive` STRING,
    `maxInclusive` STRING,
    `minLength` STRING,
    `maxLength` STRING,
    `pattern` STRING,
    `ins` STRING,
    `datatypes` STRING,
    `hasValue` STRING,
    PRIMARY KEY (id) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'iff.ngsild.flink.constraint_table'
);

In [ ]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `constraint_trigger_table`;
CREATE TABLE `constraint_trigger_table` (
    `resource` STRING,
    `event` STRING,
    `constraint_id` INTEGER,
    `triggered` BOOLEAN,
    `severity` STRING,
    `text` STRING,
    `ts` TIMESTAMP(3) METADATA FROM 'timestamp' VIRTUAL,
     WATERMARK FOR ts AS ts,
    PRIMARY KEY (resource, constraint_id, event) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'iff.ngsild.flink.constraint_trigger_table'
);

In [ ]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `constraint_combination_table`;
CREATE TABLE `constraint_combination_table` (
    `member_constraint_id` INTEGER,
    `operation` STRING,
    `target_constraint_id` INTEGER,
    PRIMARY KEY (member_constraint_id, target_constraint_id) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'iff.ngsild.flink.constraint_combination_table'
);